To train our model, we’ll use the GTZAN dataset, which is a popular dataset of 1,000 songs for music genre classification. Each song is a 30-second clip from one of 10 genres of music, spanning disco to metal. We can get the audio files and their corresponding labels from the Hugging Face Hub with the load_dataset() function from 🤗 Datasets

In [1]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Found cached dataset gtzan (C:/Users/Raj/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/a41ab02ead4abe36cdf531ccdfde57aaee72b3bda7319f3bb61eb8d8b9caebcf)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

GTZAN doesn’t provide a predefined validation set, so we’ll have to create one ourselves. The dataset is balanced across genres, so we can use the train_test_split() method to quickly create a 90/10 split as follows

In [2]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [4]:
# Let's take a look at one of the audio files
gtzan["train"][0]

{'file': 'C:\\Users\\Raj\\.cache\\huggingface\\datasets\\downloads\\extracted\\61c891d8d3b801632ad634a2a14e2e2c1a2455ea91af46e11e266c56c97fb685\\genres\\pop\\pop.00098.wav',
 'audio': {'path': 'C:\\Users\\Raj\\.cache\\huggingface\\datasets\\downloads\\extracted\\61c891d8d3b801632ad634a2a14e2e2c1a2455ea91af46e11e266c56c97fb685\\genres\\pop\\pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229]),
  'sampling_rate': 22050},
 'genre': 7}

We can also see the genre is represented as an integer, or class label, which is the format the model will make it’s predictions in. Let’s use the int2str() method of the genre feature to map these integers to human-readable names:

In [5]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

'pop'

This label looks correct, since it matches the filename of the audio file. Let’s now listen to a few more examples by using Gradio to create a simple interface with the Blocks API:

In [9]:
import gradio as gr
def generate_audio():
    example = gtzan["train"].shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"],
    ), id2label_fn(example["genre"])

with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(4):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

demo.launch(debug=True)

c:\Users\Raj\repos\HF-Audio\4-music-genre-classifier\.venv\Lib\site-packages\gradio\processing_utils.py:183: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


**Picking a pretrained model for audio classification**

Although models like Wav2Vec2 and HuBERT are very popular, we’ll use a model called DistilHuBERT. This is a much smaller (or distilled) version of the HuBERT model, which trains around 73% faster, yet preserves most of the performance.